<a href="https://colab.research.google.com/github/RagozinaMarina/question_classification/blob/master/questionClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Обновление**

С учетом исправления моей ошибки, лучше всех справляются с задачей рекуррентные сети. Сверточные сети справляются чуть хуже. Простейшая сеть по прежнему очень долго обучается. Однако качество не превышает 80 в любом случае.

## Чтение данных

In [ ]:
!git clone https://github.com/mhjabreel/CharCnn_Keras
%rm -r CharCnn_Keras

Cloning into 'CharCnn_Keras'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 168 (delta 4), reused 0 (delta 0), pack-reused 158
Receiving objects: 100% (168/168), 11.29 MiB | 23.36 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data_raw = '/content/drive/MyDrive/python_dz/train_5500.label.txt'
test_data_raw = '/content/drive/MyDrive/python_dz/TREC_10.label.txt'

In [ ]:
# читаем данные вида label text
def read_infile(infile):
    texts, labels = [], []
    with open(infile, "r",encoding='utf-8', errors='ignore') as fin:
        for line in fin:
          line = line.strip()
          if line == "":
            continue
          label, text = line.split(" ", 1)
          texts.append(text)
          labels.append(label)
    return texts, labels

In [ ]:
train_data, train_labels = read_infile(train_data_raw)
test_data, test_labels = read_infile(test_data_raw)

In [ ]:
# выделим тексты для валидации
from sklearn.model_selection import train_test_split

dev_data, train_data, dev_labels, train_labels = train_test_split(
    train_data, train_labels, train_size=1/5, stratify=train_labels, random_state = 12)

In [ ]:
from collections import Counter

print(len(Counter(train_labels)))
print(Counter(test_labels))
test_data[-1]

50
Counter({'DESC:def': 123, 'HUM:ind': 55, 'LOC:other': 50, 'NUM:date': 47, 'LOC:city': 18, 'NUM:dist': 16, 'ENTY:animal': 16, 'ENTY:substance': 15, 'NUM:other': 12, 'ENTY:other': 12, 'ENTY:color': 10, 'NUM:count': 9, 'NUM:period': 8, 'ABBR:exp': 8, 'DESC:desc': 7, 'LOC:state': 7, 'ENTY:termeq': 7, 'HUM:gr': 6, 'DESC:reason': 6, 'NUM:speed': 6, 'ENTY:currency': 6, 'ENTY:plant': 5, 'NUM:temp': 5, 'NUM:weight': 4, 'ENTY:food': 4, 'ENTY:product': 4, 'ENTY:veh': 4, 'HUM:desc': 3, 'LOC:mount': 3, 'NUM:money': 3, 'LOC:country': 3, 'NUM:perc': 3, 'ENTY:event': 2, 'DESC:manner': 2, 'ENTY:body': 2, 'ENTY:lang': 2, 'ENTY:dismed': 2, 'HUM:title': 1, 'ENTY:instru': 1, 'ABBR:abb': 1, 'ENTY:sport': 1, 'ENTY:techmeth': 1})


'What is e-coli ?'

## Подготовка данных

### Использование spacy

In [ ]:
import spacy
# локально надо было бы загрузить модель en_core_web_sm
spacy_processor = spacy.load("en_core_web_sm", disable=["parser", "ner"])

In [ ]:
def normalize_text(text, lemmatize=True, to_lower=True, transform_digits=True, only_alpha=False):
    if isinstance(text, list):
        processed_texts = list(spacy_processor.pipe(text))
        return [normalize_text(elem, lemmatize=lemmatize, to_lower=to_lower,
                               transform_digits=transform_digits, only_alpha=only_alpha) 
                for elem in processed_texts] 
    elif isinstance(text, str):
        processed_text = spacy_processor(text)
    else:  ## type(text) = Spacy.Doc
        processed_text = text
    if lemmatize:
        answer = [token.lemma_ if token.lemma_ != "-PRON-" else token.text.lower() for token in processed_text]
    else:
        answer = [token.text for token in processed_text]
    if to_lower:
        answer = [x.lower() for x in answer]
    if only_alpha:
        answer = [x for x in answer if x.isalpha() or x.isdigit()]
    if transform_digits:
        answer = ['<DIGIT>' if x.isdigit() else x for x in answer]
    return answer

нормализуем тексты

In [ ]:
train_texts = normalize_text(train_data, lemmatize=False)
dev_texts = normalize_text(dev_data, lemmatize=False)
test_texts = normalize_text(test_data, lemmatize=False)
print(test_data[-1])

What is e-coli ?


## Загрузка данных

## Преобразование данных в вектора

### Преобразование текстов в индексы

In [ ]:
from collections import defaultdict

class Vocabulary:

    def __init__(self, add_begin=True, add_end=True, min_count=1):
        self.add_begin = add_begin
        self.add_end = add_end
        self.min_count = min_count

    def fit(self, data):
        # специальные символы
        self.symbols_ = ["<PAD>", "<UNK>", "<BEGIN>", "<END>"]
        symbol_counts = defaultdict(int)
        for text in data:
            for letter in set(text):
                symbol_counts[letter] += 1
        self.symbols_ += [letter for letter, count in symbol_counts.items() if count >= self.min_count]
        self.symbol_codes_ = {letter: index for index, letter in enumerate(self.symbols_)}
        return self

    @property
    def unk(self):
        return self.symbol_codes_["<UNK>"]

    @property
    def begin(self):
        return self.symbol_codes_["<BEGIN>"]

    @property
    def end(self):
        return self.symbol_codes_["<END>"]

    def __call__(self, data):
        if isinstance(data, list) and not isinstance(data[0], str):
            return [self.__call__(text) for text in data]
        # data --- это отдельный текст
        indexes = [self.symbol_codes_.get(symbol, self.unk) for symbol in data]
        if self.add_begin:
            indexes = [self.begin] + indexes
        if self.add_end:
            indexes = indexes + [self.end]
        return indexes

преобразуем тексты

In [ ]:
vocab = Vocabulary(min_count=3).fit(train_texts)
X_train = vocab(train_texts)
X_dev = vocab(dev_texts)
X_test = vocab(test_texts)
print(f"{len(vocab.symbols_)} words in vocabulary.")
print("Example encoding", X_test[-1])

1760 words in vocabulary.
Example encoding [2, 31, 54, 1112, 196, 1, 8, 3]


Преобразуем метки.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder().fit(train_labels)
Y_train = label_encoder.transform(train_labels)
Y_dev = label_encoder.transform(dev_labels)
Y_test = label_encoder.transform(test_labels)

In [ ]:
print(test_texts[-1])
print(X_test[-1])
print(test_labels[-1], Y_test[-1])

### Загрузчик данных (DataLoader)

In [ ]:
import torch
import numpy as np
import itertools

def pad_tensor(vec, length, dim, pad_symbol):
    # vec.shape = [3, 4, 5]
    # length=7, dim=1 -> pad_size = (3, 7-4, 5)
    pad_size = list(vec.shape)
    pad_size[dim] = length - vec.shape[dim]
    answer = torch.cat([vec, torch.ones(*pad_size, dtype=torch.long) * pad_symbol], axis=dim)
    return answer

def pad_tensors(tensors, pad=0):
    # дополняет тензоры из tensors до общей максимальной длины символом pad
    tensors = [torch.LongTensor(tensor) for tensor in tensors]
    L = max(tensor.shape[0] for tensor in tensors)
    tensors = [pad_tensor(tensor, L, dim=0, pad_symbol=pad) for tensor in tensors]
    return torch.stack(tensors, axis=0)

class BatchDataLoader:

    def __init__(self, X, Y=None, batch_size=32, sort_by_length=True, state=115, device="cpu"):
        self.X = X
        self.Y = Y
        self.batch_size = batch_size
        self.sort_by_length = sort_by_length
        self.device = device
        np.random.seed(state)

    def __len__(self):
        return (len(self.X)-1) // self.batch_size + 1 

    def __iter__(self):
        if self.sort_by_length:
            # отсортировать индексы по длине объектов [1, ..., 32] -> [7, 4, 15, ...]
            order = np.argsort([len(x) for x in self.X])
            # сгруппировать в батчи [7, 4, 15, 31, 3, ...] -> [[7, 4, 15, 31], [3, ...], ...]
            batched_order = np.array([order[start:start+self.batch_size] 
                                      for start in range(0, len(self.X), self.batch_size)])
            # переупорядочить батчи случайно: [[3, 11, 21, 19], [27, ...], ..., [7, ...], ...]
            np.random.shuffle(batched_order[:-1])
            # собрать посл-ть индексов: -> [3, 11, 21, 19, 27, ...]
            self.order = np.fromiter(itertools.chain.from_iterable(batched_order), dtype=int)
        else:
            self.order = np.arange(len(self.X))
            np.random.shuffle(self.order)
        self.idx = 0
        return self

    def __next__(self):
        if self.idx >= len(self.X):
            raise StopIteration()
        end = min(self.idx + self.batch_size, len(self.X))
        indexes = [self.order[i] for i in range(self.idx, end)]
        X_to_return = pad_tensors([self.X[i] for i in indexes]).to(self.device)
        answer = {"X": X_to_return, "indexes": indexes}
        if self.Y is not None:
            Y_to_return = torch.LongTensor([self.Y[i] for i in indexes]).to(self.device)
            answer["Y"] = Y_to_return
        self.idx = end
        return answer

In [ ]:
train_dataloader = BatchDataLoader(X_train, Y_train)
dev_dataloader = BatchDataLoader(X_dev, Y_dev)
test_dataloader = BatchDataLoader(X_test)

In [ ]:
for i, batch in enumerate(train_dataloader):
    if i >= 10:
        break
    print(batch["X"].shape, batch["Y"].shape)

torch.Size([32, 14]) torch.Size([32])
torch.Size([32, 9]) torch.Size([32])
torch.Size([32, 16]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 12]) torch.Size([32])
torch.Size([32, 15]) torch.Size([32])
torch.Size([32, 13]) torch.Size([32])
torch.Size([32, 17]) torch.Size([32])
torch.Size([32, 9]) torch.Size([32])
torch.Size([32, 19]) torch.Size([32])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## Архитектура простейшей сети

Базовый класс.

In [ ]:
import torch
import torch.nn as nn

class BasicNeuralClassifier(nn.Module):

    def __init__(self, vocab_size, labels_number, device="cpu", **kwargs):
        super(BasicNeuralClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.labels_number = labels_number
        self.build_network(vocab_size, labels_number, **kwargs)
        # определяем функцию потерь
        self.criterion = nn.NLLLoss(reduction="mean")
        self.device = device
        if self.device is not None:
            self.to(self.device)
        self.optimizer = torch.optim.Adam(self.parameters())

    def build_network(self, **kwargs):
        raise NotImplementedError("You should implement network construction in your derived class.")

    def forward(self, inputs):
        raise NotImplementedError("You should implement forward pass in your derived class.")

    def train_on_batch(self, x, y):
        self.train()
        self.optimizer.zero_grad()
        loss = self._validate(x, y)
        loss["loss"].backward()
        self.optimizer.step()
        return loss

    def validate_on_batch(self, x, y):
        self.eval()
        with torch.no_grad():
            return self._validate(x, y)

    def _validate(self, x, y):
        if self.device is not None:
            x, y = x.to(self.device), y.to(self.device)
        log_probs = self(x) #   self.forward(x) = self.__call__(x)
        loss = self.criterion(log_probs, y)
        _, labels = torch.max(log_probs, dim=1)
        return {"loss": loss, "labels": labels}

In [ ]:
import torch
import torch.nn as nn

class AverageClassifier(BasicNeuralClassifier):

    def build_network(self, vocab_size, labels_number, embeddings_dim=32):
        self.embedding = nn.Embedding(vocab_size, embeddings_dim, padding_idx=0)
        self.dense = nn.Linear(embeddings_dim, labels_number)
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, inputs, hidden_inputs=None):
        if self.device is not None:
            inputs = inputs.to(self.device)
        embeddings = self.embedding(inputs)
        # покоординатное усреднение
        text_embedding = torch.mean(embeddings, axis=1)
        # финальный слой
        logits = self.dense(text_embedding)
        log_probs = self.log_softmax(logits)
        return log_probs

In [ ]:
model = AverageClassifier(vocab_size=len(vocab.symbols_), labels_number=len(label_encoder.classes_), 
                          embeddings_dim=128, device="cuda")
for name, elem in model.named_parameters():
    print(name, elem.device, elem.shape)

embedding.weight cuda:0 torch.Size([1760, 128])
dense.weight cuda:0 torch.Size([50, 128])
dense.bias cuda:0 torch.Size([50])


In [ ]:
train_batch_loader = BatchDataLoader(X_train, Y_train, batch_size=64, device="cuda:0")
for batch in train_batch_loader:
    x, y = batch["X"], batch["Y"]
    for i in range(50):
        loss = model.train_on_batch(x, y)["loss"].item()
        if i < 5 or (i+1) % 10 == 0:
            print(i, loss)
    print(model.validate_on_batch(x, y)["loss"].item())
    break

0 3.960998773574829


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


1 3.923074960708618
2 3.8855574131011963
3 3.848449468612671
4 3.811751127243042
9 3.6344149112701416
19 3.309917688369751
29 3.022268533706665
39 2.765977382659912
49 2.535027265548706
2.5130903720855713


## Обучение модели

In [ ]:
train_dataloader = BatchDataLoader(X_train, Y_train, device="cuda:0")
dev_dataloader = BatchDataLoader(X_dev, Y_dev, device="cuda:0")
test_dataloader = BatchDataLoader(X_test, device="cuda:0")
NEPOCHS = 35

In [ ]:
def update_metrics(metrics, batch_output, batch_labels):
    n_batches = metrics["n_batches"]
    metrics["loss"] = (metrics["loss"] * n_batches + batch_output["loss"].item()) / (n_batches + 1)
    metrics["n_batches"] += 1
    are_equal = (batch_output["labels"] == batch_labels).cpu().numpy().astype("int")
    metrics["correct"] += are_equal.sum()
    metrics["total"] += are_equal.shape[0]
    metrics["accuracy"] = metrics["correct"] / max(metrics["total"], 1)

In [ ]:
import tqdm

def do_epoch(model, dataloader, mode="validate", epoch=1):
    metrics = {"correct": 0, "total": 0, "loss": 0.0, "n_batches": 1}
    func = model.train_on_batch if mode == "train" else model.validate_on_batch
    progress_bar = tqdm.notebook.tqdm(dataloader, leave=True)
    progress_bar.set_description(f"{mode}, epoch={epoch}")
    for batch in progress_bar:
        batch, batch_answers = batch["X"], batch["Y"]
        batch_output = func(batch, batch_answers)
        update_metrics(metrics, batch_output, batch_answers)
        progress_bar.set_postfix({"loss": round(metrics["loss"], 4), "acc": round(100 * metrics["accuracy"], 2)})
    return metrics

In [ ]:
model = AverageClassifier(vocab_size=len(vocab.symbols_), labels_number=len(label_encoder.classes_), 
                          embeddings_dim=200, device="cuda")
best_val_acc = 0.0
checkpoint = "checkpoint_best.pt"
for epoch in range(NEPOCHS):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)
        # print("Saving ")
# model = AverageClassifier(vocab_size=len(vocab.symbols_), labels_number=len(label_encoder.classes_), 
#                           embeddings_dim=128, device="cuda")
model.load_state_dict(torch.load(checkpoint))
do_epoch(model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


{'accuracy': 0.7458715596330275,
 'correct': 813,
 'loss': 0.9808963139851888,
 'n_batches': 36,
 'total': 1090}

In [ ]:
def predict_with_model(model, X):
    model.eval()
    dataloader = BatchDataLoader(X, device=model.device)
    answer = [None] * len(X)
    for batch in dataloader:
        with torch.no_grad():
            batch_answer = model(batch["X"])
        probs = batch_answer.cpu().numpy()
        labels = probs.argmax(axis=1)
        for i, label in zip(batch["indexes"], labels):
            answer[i] = label
    return answer

In [ ]:
y_pred = predict_with_model(model, X_test)
pred_labels = label_encoder.classes_[y_pred]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
prec, rec, f1, sup = precision_recall_fscore_support(test_labels, pred_labels)
print("Точность:")
for label, x in zip(label_encoder.classes_, prec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("Полнота:")
for label, x in zip(label_encoder.classes_, rec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("F1-мера:")
for label, x in zip(label_encoder.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

Корректность: 77.40
Точность:
ABBR:abb: 100.00	ABBR:exp: 100.00	DESC:def: 80.13	DESC:desc: 44.44	DESC:manner: 50.00	DESC:reason: 85.71	ENTY:animal: 87.50	ENTY:body: 0.00	ENTY:color: 100.00	ENTY:cremat: 100.00	ENTY:currency: 33.33	ENTY:dismed: 0.00	ENTY:event: 100.00	ENTY:food: 100.00	ENTY:instru: 100.00	ENTY:lang: 17.39	ENTY:letter: 100.00	ENTY:other: 0.00	ENTY:plant: 50.00	ENTY:product: 80.00	ENTY:religion: 100.00	ENTY:sport: 60.00	ENTY:substance: 75.00	ENTY:symbol: 75.00	ENTY:techmeth: 44.44	ENTY:termeq: 88.33	ENTY:veh: 0.00	ENTY:word: 100.00	HUM:desc: 75.00	HUM:gr: 100.00	HUM:ind: 78.85	HUM:title: 36.36	LOC:city: 81.82	LOC:country: 95.74	LOC:mount: 85.71	LOC:other: 33.33	LOC:state: 75.00	NUM:code: 100.00	NUM:count: 54.55	NUM:date: 100.00	NUM:dist: 0.00	NUM:money: 100.00	
Полнота:
ABBR:abb: 100.00	ABBR:exp: 75.00	DESC:def: 98.37	DESC:desc: 57.14	DESC:manner: 100.00	DESC:reason: 100.00	ENTY:animal: 43.75	ENTY:body: 0.00	ENTY:color: 90.00	ENTY:cremat: 16.67	ENTY:currency: 50.00	ENTY:di

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Простейшая сеть медленно учится, требуется в 3,5 раза больше эпох, чтобы достичь качества как у рекуррентной сети. 

10 эпох - 69.20

35 эпох - усреднение по 5 запускам 77.80

## Загрузка предобученных весов

In [ ]:
import gensim.downloader as api

for key, data in api.info()["models"].items():
    print(key, data["description"])

fasttext-wiki-news-subwords-300 1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens).
conceptnet-numberbatch-17-06-300 ConceptNet Numberbatch consists of state-of-the-art semantic vectors (also known as word embeddings) that can be used directly as a representation of word meanings or as a starting point for further machine learning. ConceptNet Numberbatch is part of the ConceptNet open data project. ConceptNet provides lots of ways to compute with word meanings, one of which is word embeddings. ConceptNet Numberbatch is a snapshot of just the word embeddings. It is built using an ensemble that combines data from ConceptNet, word2vec, GloVe, and OpenSubtitles 2016, using a variation on retrofitting.
word2vec-ruscorpora-300 Word2vec Continuous Skipgram vectors trained on full Russian National Corpus (about 250M words). The model contains 185K words.
word2vec-google-news-300 Pre-trained vectors trained on a part of the Google News

In [ ]:
gensim_model = api.load("glove-wiki-gigaword-200")
gensim_model.vector_size

[==================================================] 100.0% 252.1/252.1MB downloaded


200

In [ ]:
new_model = AverageClassifier(vocab_size=len(vocab.symbols_), labels_number=len(label_encoder.classes_), 
                              embeddings_dim=200, device="cuda")
for key in new_model.state_dict():
    print(key, new_model.state_dict()[key].shape)


embedding.weight torch.Size([1760, 200])
dense.weight torch.Size([50, 200])
dense.bias torch.Size([50])


In [ ]:
def make_embedding_matrix(vocab, gensim_model):
    answer = np.zeros(shape=(len(vocab.symbols_), gensim_model.vector_size), dtype=float)
    for i, word in enumerate(vocab.symbols_):
        if word.lower() in gensim_model:
            answer[i] = gensim_model[word.lower()]
    return answer

In [ ]:
initial_weights = make_embedding_matrix(vocab, gensim_model)
with torch.no_grad():
    state_dict = new_model.state_dict()
    # меняем соответствующий вес в словаре
    state_dict["embedding.weight"] = torch.Tensor(initial_weights)
    # загружаем новую матрицу весов
    new_model.load_state_dict(state_dict)

In [ ]:
best_val_acc = 0.0
checkpoint = "checkpoint_best.pt"
for epoch in range(NEPOCHS):
    do_epoch(new_model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(new_model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(new_model.state_dict(), checkpoint)
        # print("Saving ")
# model = AverageClassifier(vocab_size=len(vocab.symbols_), labels_number=len(label_encoder.classes_), 
#                           embeddings_dim=128, device="cuda")
new_model.load_state_dict(torch.load(checkpoint))
do_epoch(new_model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


{'accuracy': 0.7614678899082569,
 'correct': 830,
 'loss': 0.9186626043584611,
 'n_batches': 36,
 'total': 1090}

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

y_pred = predict_with_model(new_model, X_test)
pred_labels = label_encoder.classes_[y_pred]

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
prec, rec, f1, sup = precision_recall_fscore_support(test_labels, pred_labels)
print("Точность:")
for label, x in zip(label_encoder.classes_, prec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("Полнота:")
for label, x in zip(label_encoder.classes_, rec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("F1-мера:")
for label, x in zip(label_encoder.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

Корректность: 78.00
Точность:
ABBR:abb: 100.00	ABBR:exp: 75.00	DESC:def: 80.26	DESC:desc: 50.00	DESC:manner: 66.67	DESC:reason: 71.43	ENTY:animal: 83.33	ENTY:body: 50.00	ENTY:color: 100.00	ENTY:cremat: 0.00	ENTY:currency: 0.00	ENTY:dismed: 50.00	ENTY:event: 0.00	ENTY:food: 75.00	ENTY:instru: 100.00	ENTY:lang: 100.00	ENTY:letter: 25.00	ENTY:other: 100.00	ENTY:plant: 0.00	ENTY:product: 50.00	ENTY:religion: 80.00	ENTY:sport: 100.00	ENTY:substance: 70.00	ENTY:symbol: 75.00	ENTY:techmeth: 100.00	ENTY:termeq: 30.77	ENTY:veh: 86.44	ENTY:word: 0.00	HUM:desc: 100.00	HUM:gr: 100.00	HUM:ind: 66.67	HUM:title: 79.59	LOC:city: 30.00	LOC:country: 90.00	LOC:mount: 95.83	LOC:other: 100.00	LOC:state: 50.00	NUM:code: 75.00	NUM:count: 100.00	NUM:date: 70.00	NUM:dist: 100.00	NUM:money: 100.00	NUM:ord: 100.00	
Полнота:
ABBR:abb: 100.00	ABBR:exp: 75.00	DESC:def: 99.19	DESC:desc: 57.14	DESC:manner: 100.00	DESC:reason: 83.33	ENTY:animal: 31.25	ENTY:body: 50.00	ENTY:color: 90.00	ENTY:cremat: 0.00	ENTY:currency:

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Никаких изменений от добавления предобученных векторов нет.

## Свёрточные сети

In [ ]:
import torch
import torch.nn as nn

class MultilayerConvClassifier(BasicNeuralClassifier):

    def build_network(self, vocab_size, labels_number, embeddings_dim=32,
                      n_layers=1, window=5, n_hidden=128, dropout=0.0,
                      use_batch_norm=False):
        self.n_layers = n_layers
        # n_hidden и n_window могут быть своими для каждого слоя, 
        # поэтому если передано число, мы его размножаем на все слои.
        if isinstance(n_hidden, int):
            n_hidden = [n_hidden] * self.n_layers
        self.n_hidden = n_hidden
        if isinstance(window, int):
            window = [window] * self.n_layers
        self.window = window
        self.use_batch_norm = use_batch_norm
        self.embedding = nn.Embedding(vocab_size, embeddings_dim, padding_idx=0)
        # может быть несколько слоёв свёрток
        self.convolutions = nn.ModuleList()
        for i in range(self.n_layers):
            input_dim = self.n_hidden[i-1] if i > 0 else embeddings_dim
            convolution = nn.Conv1d(input_dim, self.n_hidden[i], self.window[i], 
                                    padding=(self.window[i]-1)//2)
            layer = {
                "convolution": convolution, 
                "activation": nn.ReLU(),
                "dropout": nn.Dropout(p=dropout)
            }
            if self.use_batch_norm:
                layer["batch_norm"] = nn.BatchNorm1d(self.n_hidden[i])
            self.convolutions.append(nn.ModuleDict(layer))
        self.dense = nn.Linear(self.n_hidden[-1], labels_number)
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, inputs, hidden_inputs=None):
        if self.device is not None:
            inputs = inputs.to(self.device)
        embeddings = self.embedding(inputs)
        # для свёрточного слоя нужно сделать второй размерностью число каналов
        conv_inputs = embeddings.permute([0, 2, 1])
        for layer in self.convolutions:
            conv_outputs = layer["convolution"](conv_inputs)
            if self.use_batch_norm:
                conv_outputs = layer["batch_norm"](conv_outputs)
            conv_outputs = layer["activation"](conv_outputs)
            conv_outputs = layer["dropout"](conv_outputs)
            conv_inputs = conv_outputs
        conv_outputs = conv_outputs.permute([0, 2, 1])
        # покоординатный максимум
        max_conv_outputs, _ = torch.max(conv_outputs, dim=1)
        # финальный слой
        logits = self.dense(max_conv_outputs)
        log_probs = self.log_softmax(logits)
        return log_probs

In [ ]:
model = MultilayerConvClassifier(
            vocab_size=len(vocab.symbols_), 
            labels_number=len(label_encoder.classes_),
            embeddings_dim=200, n_hidden=200,
            n_layers=1, dropout=0.1, device="cuda", 
            use_batch_norm=True
)
for name, elem in model.named_parameters():
    print(name, elem.device, elem.shape)

embedding.weight cuda:0 torch.Size([1760, 200])
convolutions.0.convolution.weight cuda:0 torch.Size([200, 200, 5])
convolutions.0.convolution.bias cuda:0 torch.Size([200])
convolutions.0.batch_norm.weight cuda:0 torch.Size([200])
convolutions.0.batch_norm.bias cuda:0 torch.Size([200])
dense.weight cuda:0 torch.Size([50, 200])
dense.bias cuda:0 torch.Size([50])


In [ ]:
NEPOCHS = 10

model = MultilayerConvClassifier(
            vocab_size=len(vocab.symbols_), 
            labels_number=len(label_encoder.classes_),
            embeddings_dim=200, n_hidden=300,
            n_layers=1, dropout=0.1, device="cuda", 
            use_batch_norm=True
)

best_val_acc = 0.0
checkpoint = "checkpoint_best_conv.pt"
for epoch in range(NEPOCHS):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)

model.load_state_dict(torch.load(checkpoint))
do_epoch(model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


{'accuracy': 0.7422018348623853,
 'correct': 809,
 'loss': 0.9655660688877106,
 'n_batches': 36,
 'total': 1090}

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

y_pred = predict_with_model(model, X_test)
pred_labels = label_encoder.classes_[y_pred]

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
prec, rec, f1, sup = precision_recall_fscore_support(test_labels, pred_labels)
print("Точность:")
for label, x in zip(label_encoder.classes_, prec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("Полнота:")
for label, x in zip(label_encoder.classes_, rec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("F1-мера:")
for label, x in zip(label_encoder.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

Корректность: 77.40
Точность:
ABBR:abb: 33.33	ABBR:exp: 85.71	DESC:def: 79.61	DESC:desc: 50.00	DESC:manner: 50.00	DESC:reason: 100.00	ENTY:animal: 72.73	ENTY:body: 0.00	ENTY:color: 100.00	ENTY:cremat: 0.00	ENTY:currency: 100.00	ENTY:dismed: 0.00	ENTY:event: 100.00	ENTY:food: 66.67	ENTY:instru: 100.00	ENTY:lang: 100.00	ENTY:letter: 31.25	ENTY:other: 0.00	ENTY:plant: 0.00	ENTY:product: 33.33	ENTY:religion: 57.14	ENTY:sport: 100.00	ENTY:substance: 50.00	ENTY:symbol: 100.00	ENTY:techmeth: 100.00	ENTY:termeq: 80.00	ENTY:veh: 81.25	ENTY:word: 0.00	HUM:desc: 87.50	HUM:gr: 100.00	HUM:ind: 50.00	HUM:title: 90.24	LOC:city: 55.56	LOC:country: 90.00	LOC:mount: 97.83	LOC:other: 90.00	LOC:state: 40.00	NUM:code: 46.15	NUM:count: 100.00	NUM:date: 63.64	NUM:dist: 100.00	NUM:money: 100.00	NUM:ord: 100.00	
Полнота:
ABBR:abb: 100.00	ABBR:exp: 75.00	DESC:def: 98.37	DESC:desc: 71.43	DESC:manner: 100.00	DESC:reason: 83.33	ENTY:animal: 50.00	ENTY:body: 0.00	ENTY:color: 80.00	ENTY:cremat: 0.00	ENTY:currency: 3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Усреденение по 10 эпохам за 5 запусков: 77.50


## Рекуррентные сети

In [ ]:
class RNNClassifier(BasicNeuralClassifier):

    def build_network(self, vocab_size, labels_number, embeddings_dim=32,
                      n_hidden=64, dropout=0.0, mean=False,
                      device="cpu"):
        self.n_hidden = n_hidden
        self.embedding = nn.Embedding(vocab_size, embeddings_dim, padding_idx=0)
        self.rnn = nn.GRU(embeddings_dim, n_hidden, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.dense = nn.Linear(2*n_hidden, labels_number)
        self.log_softmax = nn.LogSoftmax(dim=-1)
        # финальное состояние считается через усреднение
        self.mean = mean

    def forward(self, inputs):
        if self.device is not None:
            inputs = inputs.to(self.device)
        embeddings = self.embedding(inputs)
        # rnn_outputs.shape = (B, L, 2*H), state.shape = (2, B, H)
        rnn_outputs, state = self.rnn(embeddings)
        if self.mean:
            state = torch.mean(rnn_outputs, dim=1)
        else:
            # state.shape = (B, 2, H)
            state = state.permute((1, 0, 2))
            # state.shape = (B, 2*H)
            state = state.reshape(-1, 2*self.n_hidden)
        state = self.dropout(state)
        # финальный слой
        logits = self.dense(state)
        log_probs = self.log_softmax(logits)
        return log_probs

In [ ]:
model = RNNClassifier(
            vocab_size=len(vocab.symbols_), 
            labels_number=len(label_encoder.classes_),
            embeddings_dim=200, n_hidden=128,
            dropout=0.2, device="cuda",
            mean=True
)
for name, elem in model.named_parameters():
    print(name, elem.device, elem.shape)

embedding.weight cuda:0 torch.Size([1760, 200])
rnn.weight_ih_l0 cuda:0 torch.Size([384, 200])
rnn.weight_hh_l0 cuda:0 torch.Size([384, 128])
rnn.bias_ih_l0 cuda:0 torch.Size([384])
rnn.bias_hh_l0 cuda:0 torch.Size([384])
rnn.weight_ih_l0_reverse cuda:0 torch.Size([384, 200])
rnn.weight_hh_l0_reverse cuda:0 torch.Size([384, 128])
rnn.bias_ih_l0_reverse cuda:0 torch.Size([384])
rnn.bias_hh_l0_reverse cuda:0 torch.Size([384])
dense.weight cuda:0 torch.Size([50, 256])
dense.bias cuda:0 torch.Size([50])


Выучиваем один батч

In [ ]:
train_batch_loader = BatchDataLoader(X_train, Y_train, batch_size=64, device="cuda:0")
model = RNNClassifier(
            vocab_size=len(vocab.symbols_), 
            labels_number=len(label_encoder.classes_),
            embeddings_dim=200, n_hidden=128,
            dropout=0.1, device="cuda",
            mean=True
)
for batch in train_batch_loader:
    x, y = batch["X"], batch["Y"]
    for i in range(50):
        loss = model.train_on_batch(x, y)["loss"].item()
        if i < 5 or (i+1) % 10 == 0:
            print(i, loss)
    print(model.validate_on_batch(x, y)["loss"].item())
    break

0 3.915283203125
1 3.7732081413269043
2 3.645939588546753
3 3.491917133331299
4 3.3615946769714355
9 2.7319626808166504
19 1.8839913606643677
29 1.1154853105545044
39 0.5208755731582642
49 0.20588701963424683
0.17837226390838623


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


### Тестирование модели

In [ ]:
train_dataloader = BatchDataLoader(X_train, Y_train, device="cuda:0")
dev_dataloader = BatchDataLoader(X_dev, Y_dev, device="cuda:0")
test_dataloader = BatchDataLoader(X_test, device="cuda:0")

NEPOCHS = 10

model = RNNClassifier(
            vocab_size=len(vocab.symbols_), 
            labels_number=len(label_encoder.classes_),
            embeddings_dim=200, n_hidden=128,
            dropout=0.2, device="cuda",
            mean=True
)

best_val_acc = 0.0
checkpoint = "checkpoint_best_gru.pt"
for epoch in range(NEPOCHS):
    do_epoch(model, train_dataloader, mode="train", epoch=epoch+1)
    epoch_metrics = do_epoch(model, dev_dataloader, mode="validate", epoch=epoch+1)
    if epoch_metrics["accuracy"] > best_val_acc:
        best_val_acc = epoch_metrics["accuracy"]
        torch.save(model.state_dict(), checkpoint)

model.load_state_dict(torch.load(checkpoint))
do_epoch(model, dev_dataloader, mode="validate", epoch="evaluate")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


{'accuracy': 0.7724770642201835,
 'correct': 842,
 'loss': 0.9411125151802682,
 'n_batches': 36,
 'total': 1090}

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

y_pred = predict_with_model(model, X_test)
pred_labels = label_encoder.classes_[y_pred]

accuracy = accuracy_score(test_labels, pred_labels)
print("Корректность: {:.2f}".format(100 * accuracy))
prec, rec, f1, sup = precision_recall_fscore_support(test_labels, pred_labels)
print("Точность:")
for label, x in zip(label_encoder.classes_, prec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("Полнота:")
for label, x in zip(label_encoder.classes_, rec):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")
print("F1-мера:")
for label, x in zip(label_encoder.classes_, f1):
    print("{}: {:.2f}".format(label, 100*x), end="\t")
print("")

Корректность: 79.20
Точность:
ABBR:abb: 33.33	ABBR:exp: 100.00	DESC:def: 84.29	DESC:desc: 80.00	DESC:manner: 66.67	DESC:reason: 66.67	ENTY:animal: 87.50	ENTY:body: 0.00	ENTY:color: 100.00	ENTY:cremat: 100.00	ENTY:currency: 50.00	ENTY:dismed: 0.00	ENTY:event: 50.00	ENTY:food: 100.00	ENTY:instru: 100.00	ENTY:lang: 23.81	ENTY:letter: 0.00	ENTY:other: 0.00	ENTY:plant: 33.33	ENTY:product: 50.00	ENTY:religion: 0.00	ENTY:sport: 50.00	ENTY:substance: 53.85	ENTY:symbol: 100.00	ENTY:techmeth: 66.67	ENTY:termeq: 36.36	ENTY:veh: 91.53	ENTY:word: 0.00	HUM:desc: 100.00	HUM:gr: 100.00	HUM:ind: 100.00	HUM:title: 76.79	LOC:city: 62.50	LOC:country: 90.00	LOC:mount: 93.88	LOC:other: 100.00	LOC:state: 100.00	NUM:code: 83.33	NUM:count: 66.67	NUM:date: 63.64	NUM:dist: 100.00	NUM:money: 100.00	NUM:ord: 100.00	
Полнота:
ABBR:abb: 100.00	ABBR:exp: 75.00	DESC:def: 95.93	DESC:desc: 57.14	DESC:manner: 100.00	DESC:reason: 100.00	ENTY:animal: 43.75	ENTY:body: 0.00	ENTY:color: 100.00	ENTY:cremat: 16.67	ENTY:currency

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Усреднение за 10 эпох по 5 запускам: 79.00